In [1]:
import pandas as pd 

In [2]:
pip install pandas pyodbc sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-S9QQ7CU\\MSSQLSERVER2;'
    'DATABASE=pipelines;'
    'UID=sa;'
    'PWD=Keeru@1026;'
    'TrustServerCertificate=yes;'
)

print("✅ Connected successfully!")


✅ Connected successfully!


In [4]:
df = pd.read_csv(r'C:\Users\Keerthi\Downloads\us_customer_data 1.csv')  # Update path
print(df.head()) 

   customer_id              name                        email         phone  \
0            1     Michelle Kidd           vayala@example.net  619-723-4258   
1            2       Brad Newton  taylorcatherine@example.net  537-674-1158   
2            3      Larry Torres         dsanchez@example.net  810-256-4505   
3            4    Kimberly Price    jessicaknight@example.com  423-222-9779   
4            5  Matthew Phillips        qwilliams@example.com  220-763-3522   

                                             address registration_date  \
0                        USNS Santiago, FPO AE 80872        25-01-2025   
1  38783 Oliver Street, West Kristenborough, MT 9...        13-07-2023   
2     6845 Steele Turnpike, West Erikabury, UT 37487        18-08-2023   
3         1631 Alexis Meadows, Lake Amanda, CA 75179        08-12-2024   
4  2274 Williams Heights Suite 895, Andersonhaven...        03-02-2024   

  loyalty_status  
0           Gold  
1         Silver  
2         Bronze  
3   

In [5]:
orders_df = pd.read_csv(r"C:\Users\Keerthi\Downloads\order_data 1.csv")
print(orders_df.head())

                               order_id  customer_id  order_date  \
0  892a07a4-d252-4775-85e0-73077143e1c6          966  2024-11-27   
1  ae160758-e187-47b2-9350-032f88f55491          345  2023-03-27   
2  7c50456e-6123-45cc-aa19-128bef3754d6          503  2024-03-31   
3  c788b56b-3716-4cd9-a827-d4dc401ba00c          385  2023-08-09   
4  925ab5b1-adb1-4302-a70b-1c2db724e02b          817  2023-04-19   

   order_amount order_status product_category  
0        317.64    Cancelled    Home & Garden  
1        645.87    Completed    Home & Garden  
2        880.86      Pending         Clothing  
3        876.83    Cancelled    Home & Garden  
4        264.53      Pending    Home & Garden  


In [6]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import pandas as pd

# ✅ Sample DataFrame (remove this if you already have df)
# df = pd.read_csv("your_file.csv")

# ✅ Create a secure connection URL using SQLAlchemy
connection_url = URL.create(
    "mssql+pyodbc",
    username="sa",
    password="Keeru@1026",
    host="DESKTOP-S9QQ7CU\\MSSQLSERVER2",
    database="pipelines",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "TrustServerCertificate": "yes"
    }
)

# ✅ Create the engine
engine = create_engine(connection_url)

# ✅ Upload df to SQL Server in the table named 'pipelines'
df.to_sql("customers", con=engine, if_exists="replace", index=False)
orders_df.to_sql("orders", con=engine, if_exists="replace", index=False)

print("✅ Data uploaded successfully to SQL Server!")

✅ Data uploaded successfully to SQL Server!


In [7]:
engine = create_engine(connection_url)

In [8]:
customers_df = pd.read_sql("SELECT * FROM customers", con=engine)

In [9]:
print(customers_df.head())

   customer_id              name                        email         phone  \
0            1     Michelle Kidd           vayala@example.net  619-723-4258   
1            2       Brad Newton  taylorcatherine@example.net  537-674-1158   
2            3      Larry Torres         dsanchez@example.net  810-256-4505   
3            4    Kimberly Price    jessicaknight@example.com  423-222-9779   
4            5  Matthew Phillips        qwilliams@example.com  220-763-3522   

                                             address registration_date  \
0                        USNS Santiago, FPO AE 80872        25-01-2025   
1  38783 Oliver Street, West Kristenborough, MT 9...        13-07-2023   
2     6845 Steele Turnpike, West Erikabury, UT 37487        18-08-2023   
3         1631 Alexis Meadows, Lake Amanda, CA 75179        08-12-2024   
4  2274 Williams Heights Suite 895, Andersonhaven...        03-02-2024   

  loyalty_status  
0           Gold  
1         Silver  
2         Bronze  
3   

In [10]:
orders_df =pd.read_sql("select * from orders", con=engine)

In [11]:
print(orders_df.head())

                               order_id  customer_id  order_date  \
0  892a07a4-d252-4775-85e0-73077143e1c6          966  2024-11-27   
1  ae160758-e187-47b2-9350-032f88f55491          345  2023-03-27   
2  7c50456e-6123-45cc-aa19-128bef3754d6          503  2024-03-31   
3  c788b56b-3716-4cd9-a827-d4dc401ba00c          385  2023-08-09   
4  925ab5b1-adb1-4302-a70b-1c2db724e02b          817  2023-04-19   

   order_amount order_status product_category  
0        317.64    Cancelled    Home & Garden  
1        645.87    Completed    Home & Garden  
2        880.86      Pending         Clothing  
3        876.83    Cancelled    Home & Garden  
4        264.53      Pending    Home & Garden  


In [12]:
customers_df[['name']].loc[132]

name    Elizabeth Williams MD
Name: 132, dtype: object

In [13]:
#split full name into two parts first and last 
prefixes = r'^(Mr\.|Mrs\.|Miss|Ms\.|Dr\.)\s*'
suffixes = r'\s*(Jr\.|Sr\.|II|III|DDS|PhD|MD|MBA|DVM|Esq\.?)$'
customers_df['name'] = customers_df['name'].str.replace(prefixes, '', regex=True)
customers_df['name'] = customers_df['name'].str.replace(suffixes, '', regex=True)
customers_df[['name']].loc[572]

name    Philip Wood
Name: 572, dtype: object

In [14]:
customers_df[['first_name', 'last_name']] = customers_df['name'].str.strip().str.split(' ', n=1, expand=True)
print(customers_df[['first_name', 'last_name']])

    first_name last_name
0     Michelle      Kidd
1         Brad    Newton
2        Larry    Torres
3     Kimberly     Price
4      Matthew  Phillips
..         ...       ...
995      Jerry  Mcdaniel
996       Jodi   Simpson
997    Crystal     Brown
998    Gregory    Duarte
999      Jason     Walls

[1000 rows x 2 columns]


In [15]:
customers_df

,customer_id,name,email,phone,address,registration_date,loyalty_status,first_name,last_name
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,Kimberly,Price
4,5,Matthew Phillips,qwilliams@example.com,220-763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,Matthew,Phillips
...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,walkerlisa@example.net,638-989-9441,"34746 Smith Gateway, New Sarah, AS 12715",10-02-2025,Silver,Jerry,Mcdaniel
996,997,Jodi Simpson,eric24@example.org,483-625-2940,"2876 Tucker Road Suite 947, North Tommyborough...",18-04-2024,Bronze,Jodi,Simpson
997,998,Crystal Brown,pshaffer@example.net,390-747-3088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",30-08-2024,Bronze,Crystal,Brown
998,999,Gregory Duarte,caitlindunlap@example.org,257-409-8196,"Unit 6377 Box 7662, DPO AP 03300",16-05-2024,Gold,Gregory,Duarte


In [16]:
status_mapping = {
    'Gold': '2',
    'Silver': '1',
    'Bronze': '0'
}

In [18]:
customers_df['customer_status_mapping'] = customers_df['loyalty_status'].map(status_mapping)

In [19]:
customers_df

,customer_id,name,email,phone,address,registration_date,loyalty_status,first_name,last_name,customer_status_mapping
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd,2
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton,1
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres,0
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,Kimberly,Price,2
4,5,Matthew Phillips,qwilliams@example.com,220-763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,Matthew,Phillips,2
...,...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,walkerlisa@example.net,638-989-9441,"34746 Smith Gateway, New Sarah, AS 12715",10-02-2025,Silver,Jerry,Mcdaniel,1
996,997,Jodi Simpson,eric24@example.org,483-625-2940,"2876 Tucker Road Suite 947, North Tommyborough...",18-04-2024,Bronze,Jodi,Simpson,0
997,998,Crystal Brown,pshaffer@example.net,390-747-3088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",30-08-2024,Bronze,Crystal,Brown,0
998,999,Gregory Duarte,caitlindunlap@example.org,257-409-8196,"Unit 6377 Box 7662, DPO AP 03300",16-05-2024,Gold,Gregory,Duarte,2


In [20]:
customers_df['country_codes'] = customers_df['address'].str.slice(-8,-6)

In [21]:
customers_df

,customer_id,name,email,phone,address,registration_date,loyalty_status,first_name,last_name,customer_status_mapping,country_codes
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,Michelle,Kidd,2,AE
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,Brad,Newton,1,MT
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,Larry,Torres,0,UT
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,Kimberly,Price,2,CA
4,5,Matthew Phillips,qwilliams@example.com,220-763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,Matthew,Phillips,2,OR
...,...,...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,walkerlisa@example.net,638-989-9441,"34746 Smith Gateway, New Sarah, AS 12715",10-02-2025,Silver,Jerry,Mcdaniel,1,AS
996,997,Jodi Simpson,eric24@example.org,483-625-2940,"2876 Tucker Road Suite 947, North Tommyborough...",18-04-2024,Bronze,Jodi,Simpson,0,AL
997,998,Crystal Brown,pshaffer@example.net,390-747-3088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",30-08-2024,Bronze,Crystal,Brown,0,DE
998,999,Gregory Duarte,caitlindunlap@example.org,257-409-8196,"Unit 6377 Box 7662, DPO AP 03300",16-05-2024,Gold,Gregory,Duarte,2,AP


In [22]:
customers_df['phone'] = '+1' + customers_df['phone']

In [23]:
customers_df[['phone']]

,phone
0,+1619-723-4258
1,+1537-674-1158
2,+1810-256-4505
3,+1423-222-9779
4,+1220-763-3522
...,...
995,+1638-989-9441
996,+1483-625-2940
997,+1390-747-3088
998,+1257-409-8196


In [24]:
customers_df['phone']= customers_df['phone'].str.replace('-', '')

In [25]:
customers_df[['phone']]

,phone
0,+16197234258
1,+15376741158
2,+18102564505
3,+14232229779
4,+12207633522
...,...
995,+16389899441
996,+14836252940
997,+13907473088
998,+12574098196
